In [1]:
import os
cache_dir = '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache'
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME']=cache_dir
os.environ['HF_HUB_CACHE']=cache_dir+'/hub'

In [2]:
import torch
from transformers import pipeline, AutoTokenizer
# import multiprocessing as mp

# mp.set_start_method("spawn", force=True)

from datasets import load_from_disk
from vllm.inputs import TokensPrompt
from vllm import LLM, SamplingParams
from tqdm import tqdm



/work/pi_mccallum_umass_edu/kchimmad_umass_edu/conda_envs/reason/lib/python3.12/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
# Loading data
data_path= "../datasets/gec/tokenized/LLaMA3B-Instruct/"
split='test'
data=load_from_disk(data_path+split+'/0-shot')
# data = load_from_disk(f"{data_path}/tokenized/LLaMA8B-Instruct/{split}/0-shot/")
data

Dataset({
    features: ['id', 'input', 'edits', 'output', 'input_ids'],
    num_rows: 1312
})

In [4]:
# Loading model
hf_token = os.getenv("hf_token")

model_name= "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, cache_dir=cache_dir)
tokenizer.pad_token_id = tokenizer.eos_token_id



In [5]:
kwargs={
    'download_dir':cache_dir,
    'max_model_len': 2000
}

model = LLM(
        model=model_name,  
        tensor_parallel_size=1,
        trust_remote_code=True,
        # gpu_memory_utilization=0.97,  # Increase this value
    **kwargs
) 



Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-3B-Instruct/.no_exist/0cb88a4f764b7a12671c53f0838cd831a0843b95/preprocessor_config.json'


INFO 05-07 23:53:57 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2000, download_dir='/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache', load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llama/Llama-3.2-3B-Instruct, 

Ignored error while writing commit hash to /scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-3B-Instruct/refs/main: [Errno 13] Permission denied: '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-3B-Instruct/refs/main'.


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 05-07 23:53:59 model_runner.py:926] Loading model weights took 6.0160 GB
INFO 05-07 23:54:00 gpu_executor.py:122] # GPU blocks: 18966, # CPU blocks: 2340
INFO 05-07 23:54:03 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-07 23:54:03 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 05-07 23:54:14 model_runner.py:1335] Graph capturing finished in 11 secs.


 94%|█████████▍| 1236/1312 [01:31<00:00, 109.36it/s]

In [6]:
stop_strings =  ['<|eot_id|>', '<|start_header_id|>user<|end_header_id|>', 'Q:', '</s>', '<|im_end|>']


sampling_params = SamplingParams(n=1,
                                 temperature=0,
                                 max_tokens=500,
                                 stop=stop_strings,
                                 # logprobs=2,
                                 best_of=1,
                                 presence_penalty=0,
                                 top_p=1,
                                 top_k=1
                                )

In [7]:
# import json
# task='gec'
# task_prompt_path=f'../prompts/{task}.json'
# with open(task_prompt_path) as fp:
#     task_prompt = json.load(fp)
# task_prompt

In [8]:
# prompt=[]
# for i in range(len(data)):
# # for i in range(26,28):
#     chat=[
#         {
#             "role":"system",
#             "content":task_prompt['system_msg'],
#         },
#         {
#             "role":"user",
#             "content":task_prompt['user_msg'].format(instruction=task_prompt['task_prompt'], question=data[i]['input'])
#         }
#     ]
#     prompt.append(tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True))
# # print(prompt)

In [9]:
# print(prompt[332])

In [10]:
all_outputs = model.generate(data['input_ids'], sampling_params)

Processed prompts: 100%|██████████| 1312/1312 [00:38<00:00, 33.73it/s, est. speed input: 4611.07 toks/s, output: 3023.16 toks/s] 
